# 概述
区域协调，路网固定为2*2矩阵，索引$i, j\in [0, 1]$    
路网所有交叉口采用单口放行   
其中干道的协调方法使用改进型数解法      
交叉口距离用二维向量表示为$(d_{(i-1, j)\to(i, j)}, d_{(i, j-1)\to(i, j)})$

In [11]:
import pandas as pd
import numpy as np

# parameters

# dis_matrix: each element is a pair of distances
# first is the dis from current to left, second is the dis from current to up
area_dis_matrix  =  [[(0, 0), (219, 0)], 
                [(0, 231), (213, 239)]] 

# each element in matrix is a tuple with 4 vals
# sequence of tuple: SNEW
area_ratio_matrix = [[(0.27, 0.27, 0.23, 0.23), (0.49, 0.0, 0.19, 0.19)],
                    [(0.24, 0.3, 0.26, 0.2), (0.35, 0.0, 0.0, 0.53)]] 

c_min = 78
c_max = 120
v = 12.5    # unit: m/s
step = 2
relaxation = 0.1

'''
phase: use all phase sequence
        Valid phase sequence 
        for N-S:
            0: NSWE
            1: SNEW
            2: NESW
            3: NWSE
        for W-E: 
            0: WESN (SNWE)
            1: EWNS (NSEW)
            2: WSEN (NWSE)
            3: WNES (NESW)
        for W-N:
            0: WNSE (NSEW)
            1: NWSE 
            2: WSNE (SNEW)
            3: WENS (NSWE)

        for all, valid phase sequence:
            0: NSWE 
            1: SNEW
            2: NESW
            3: NWSE
            4: NSEW (EWNS for W-E line)
            5: SNWE (WESN for W-E line)
'''

'\nphase: use all phase sequence\n        Valid phase sequence \n        for N-S:\n            0: NSWE\n            1: SNEW\n            2: NESW\n            3: NWSE\n        for W-E: \n            0: WESN (SNWE)\n            1: EWNS (NSEW)\n            2: WSEN (NWSE)\n            3: WNES (NESW)\n        for W-N:\n            0: WNSE (NSEW)\n            1: NWSE \n            2: WSNE (SNEW)\n            3: WENS (NSWE)\n\n        for all, valid phase sequence:\n            0: NSWE \n            1: SNEW\n            2: NESW\n            3: NWSE\n            4: NSEW (EWNS for W-E line)\n            5: SNWE (WESN for W-E line)\n'

# 函数库

In [12]:
def getInterval(c):
    return 0.5 * v * c

def getDeltaT(pos, seq_idx, c, direction):
    t_i = (1 - np.sum(area_ratio_matrix[pos[0]][pos[1]])) / len(area_ratio_matrix[pos[0]][pos[1]])

    if direction == 'ns':
        delta_t = 0.5*area_ratio_matrix[pos[0]][pos[1]][0]*c + 0.5*area_ratio_matrix[pos[0]][pos[1]][1]*c

        if seq_idx == 0:
            delta_t += t_i
        elif seq_idx == 1:
            delta_t = -1 * delta_t - t_i
        elif seq_idx == 2:
            delta_t += area_ratio_matrix[pos[0]][pos[1]][2]*c + 2*t_i
        elif seq_idx == 3:
            delta_t += area_ratio_matrix[pos[0]][pos[1]][3]*c + 2*t_i

    elif direction == 'we':
        delta_t = 0.5*area_ratio_matrix[pos[0]][pos[1]][3]*c + 0.5*area_ratio_matrix[pos[0]][pos[1]][2]*c

        if seq_idx == 0:
            delta_t += t_i
        elif seq_idx == 1:
            delta_t = -1 * delta_t - t_i
        elif seq_idx == 2:
            delta_t += area_ratio_matrix[pos[0]][pos[1]][0]*c + 2*t_i
        elif seq_idx == 3:
            delta_t += area_ratio_matrix[pos[0]][pos[1]][1]*c + 2*t_i

    elif direction == 'wn':
        delta_t = 0.5*area_ratio_matrix[pos[0]][pos[1]][3]*c + 0.5*area_ratio_matrix[pos[0]][pos[1]][1]*c

        if seq_idx == 0:
            delta_t += t_i
        elif seq_idx == 1:
            delta_t = -1*delta_t + t_i
        elif seq_idx == 2:
            delta_t += 0.5*area_ratio_matrix[pos[0]][pos[1]][0]*c + 2*t_i
        elif seq_idx == 3:
            delta_t += 0.5*area_ratio_matrix[pos[0]][pos[1]][2]*c + 2*t_i

    return delta_t

def convertBaseSeqIndex(direction, seq):

    res = -1

    if direction == 'we':
        if seq == 0:
            res = 0
        elif seq == 1:
            res = 1
        elif seq == 2:
            res = 3
        elif seq == 3:
            res = 2
        elif seq == 4:
            res = 1
        elif seq == 5:
            res = 0
    elif direction == 'ns':
        if seq in range(4):
            res = seq
        elif seq == 4:
            res = 0
        elif seq == 5:
            res = 1
    elif direction == 'wn':
        if seq == 0:
            res = 3
        elif seq == 1:
            res = 2
        elif seq == 2:
            res = 0
        elif seq == 3:
            res = 2
        elif seq == 4:
            res = 0
        elif seq == 5:
            res = 1

    return res

def getIdealDis(a_pos, b_pos, a_seq_idx, b_seq_idx, c, direction):
    # FIXME: error in caclulating delta_t_a
    delta_t_a = getDeltaT(a_pos, convertBaseSeqIndex(direction, a_seq_idx), c, direction)
    delta_t_b = getDeltaT(b_pos, convertBaseSeqIndex(direction, b_seq_idx), c, direction)

    res = 0.5*v*delta_t_a - 0.5*v*delta_t_b
    while res < 0:
        res += getInterval(c)

    return res

def getSRI(idea_dis, real_dis, c, pos=True):
    diff = real_dis - idea_dis
    while np.abs(diff) > np.abs(diff - getInterval(c)):
        diff -= getInterval(c)
        idea_dis += getInterval(c)

    if pos:
        return diff, idea_dis 
    else:
        return diff

def getOffset(idea_dis, real_dis, ratio, c):
    diff, idea_pos = getSRI(idea_dis, idea_pos, c)
    offset = idea_pos/v - 0.5*c*ratio
    while offset > c:
        offset -= c
    while offset < 0:
        offset += c

# 相对基准交叉口
计算 bias split

In [13]:
base_phase = 0    # phase of absolute benchmark

relative_table = []

cols = ['cycle']
cols += ['WE' + str(i) for i in range(6)]
cols += ['NS' + str(i) for i in range(6)]
cols.append('interval')

for t in range(c_min, c_max+step, step):
    table_row = [t]
    interval = getInterval(t)

    for i in range(6):
        we_idea_dis = getIdealDis((0, 0), (0, 1), base_phase, i, t, 'we')
        we_dis = area_dis_matrix[0][1][0]
        we_bias = we_dis % interval
        we_diff = we_bias - we_idea_dis
        #print("idea: {}, real: {}, diff: {}".format(we_idea_dis, we_bias, we_diff))
        while np.abs(we_diff - interval) < np.abs(we_diff):
            we_diff -= interval
            #print(we_diff)
        while we_diff < 0 and np.abs(we_diff + interval) < np.abs(we_diff):
            we_diff += interval
            #print(we_diff)
        we_split = 0.5 * we_diff / interval

        table_row.append(we_split)

    for i in range(6):
        ns_idea_dis = getIdealDis((0, 0), (1, 0), base_phase, i, t, 'ns')
        ns_dis = area_dis_matrix[1][0][1]
        ns_bias = ns_dis % interval
        ns_diff = ns_bias - ns_idea_dis
        while np.abs(ns_diff - interval) < np.abs(ns_diff):
            ns_diff -= interval
        while ns_diff < 0 and np.abs(ns_diff + interval) < np.abs(ns_diff):
            ns_diff += interval
        ns_split = 0.5 * ns_diff / interval

        table_row.append(ns_split)

    table_row.append(interval)
    relative_table.append(table_row)

relative_df = pd.DataFrame(relative_table, columns=cols)
#relative_df.to_excel('relative.xlsx')
relative_df

,cycle,WE0,WE1,WE2,WE3,WE4,WE5,NS0,NS1,NS2,NS3,NS4,NS5,interval
0,78,0.204824,0.014407,0.205032,-0.049968,0.014407,0.204824,0.236923,-0.033077,-0.133077,-0.163077,0.236923,-0.033077,487.5
1,80,0.199203,0.008797,0.199406,-0.055594,0.008797,0.199203,0.231000,-0.039000,-0.139000,-0.169000,0.231000,-0.039000,500.0
2,82,0.193857,0.003460,0.194055,-0.060945,0.003460,0.193857,0.225366,-0.044634,-0.144634,-0.174634,0.225366,-0.044634,512.5
3,84,0.188765,-0.001622,0.188958,-0.066042,-0.001622,0.188765,0.220000,-0.050000,-0.150000,-0.180000,0.220000,-0.050000,525.0
4,86,0.183910,-0.006468,0.184099,-0.070901,-0.006468,0.183910,0.214884,-0.055116,-0.155116,-0.185116,0.214884,-0.055116,537.5
5,88,0.179276,-0.011094,0.179460,-0.075540,-0.011094,0.179276,0.210000,-0.060000,-0.160000,-0.190000,0.210000,-0.060000,550.0
6,90,0.174847,-0.015514,0.175028,-0.079972,-0.015514,0.174847,0.205333,-0.064667,-0.164667,-0.194667,0.205333,-0.064667,562.5
7,92,0.170611,-0.019742,0.170788,-0.084212,-0.019742,0.170611,0.200870,-0.069130,-0.169130,-0.199130,0.200870,-0.069130,575.0
8,94,0.166556,-0.023790,0.166729,-0.088271,-0.023790,0.166556,0.196596,-0.073404,-0.173404,-0.203404,0.196596,-0.073404,587.5
9,96,0.162669,-0.027669,0.162839,-0.092161,-0.027669,0.162669,0.192500,-0.077500,-0.177500,-0.207500,0.192500,-0.077500,600.0


筛选合格周期及相位

In [14]:
qualified_table = []

qualified_cols = ['cycle', 'WE', 'NS']

for idx, data in relative_df.iterrows():
    table_row = [data['cycle']]
    phase_list = []
    for i in range(6):
        series_idx = 'WE' + str(i)
        if np.abs(data[series_idx]) <= relaxation:
            phase_list.append(i)
    if not phase_list:
        continue
    table_row.append(phase_list)

    phase_list = []
    for i in range(6):
        series_idx = 'NS' + str(i)
        if np.abs(data[series_idx]) <= relaxation:
            phase_list.append(i)
    if not phase_list:
        continue
    table_row.append(phase_list)

    qualified_table.append(table_row)

qualified_df = pd.DataFrame(qualified_table, columns=qualified_cols)
#qualified_df.to_excel('qualified.xlsx')
qualified_df

,cycle,WE,NS
0,78.0,"[1, 3, 4]","[1, 5]"
1,80.0,"[1, 3, 4]","[1, 5]"
2,82.0,"[1, 3, 4]","[1, 5]"
3,84.0,"[1, 3, 4]","[1, 5]"
4,86.0,"[1, 3, 4]","[1, 5]"
5,88.0,"[1, 3, 4]","[1, 5]"
6,90.0,"[1, 3, 4]","[1, 5]"
7,92.0,"[1, 3, 4]","[1, 5]"
8,94.0,"[1, 3, 4]","[1, 5]"
9,96.0,"[1, 3, 4]","[1, 5]"


# 非基准交叉口

In [15]:
scheme = qualified_df.iloc[0]
t = scheme['cycle']
seq_we = scheme['WE']
seq_ns = scheme['NS']

dt_zero_ns = getDeltaT((0, 0), base_phase, t, 'ns')
dt_zero_we = getDeltaT((0, 0), base_phase, t, 'we')
dt_zero_wn = getDeltaT((0, 0), base_phase, t, 'wn')

min_bias_split = float('inf')
optimal_seq = []
for phase_we in seq_we:
    dt_we_ns = getDeltaT((0, 1), phase_we, t, 'ns')
    dt_we_we = getDeltaT((0, 1), phase_we, t, 'we')
    dt_we_wn = getDeltaT((0, 1), phase_we, t, 'wn')

    offset_we = 0.5*area_ratio_matrix[0][0][3]*t + \
                getIdealDis((0, 0), (0, 1), base_phase, phase_we, t, 'we')/v - \
                0.5*area_ratio_matrix[0][1][3]*t
    offset_we = offset_we % t
    '''
    offset_we = getSRI(getIdealDis((0, 0), (0, 1), base_phase, phase_we, t, 'we'), 
                        area_dis_matrix[0][1][0], t, False) / v - \
                0.5 * t * area_ratio_matrix[0][1][3]
    '''

    for phase_ns in seq_ns:
        dt_ns_ns = getDeltaT((1, 0), phase_ns, t, 'ns')
        dt_ns_we = getDeltaT((1, 0), phase_ns, t, 'we')
        dt_ns_wn = getDeltaT((1, 0), phase_ns, t, 'wn')

        for phase_non in range(6):
            dt_non_ns = getDeltaT((1, 1), phase_non, t, 'ns')
            dt_non_we = getDeltaT((1, 1), phase_non, t, 'we')
            dt_non_wn = getDeltaT((1, 1), phase_non, t, 'wn')

            offset_diff_we_ns = 0.5*(dt_zero_ns-dt_ns_ns+dt_ns_we-dt_non_we) - \
                                0.5*(dt_zero_we-dt_we_we+dt_we_ns-dt_non_ns) + \
                                    (dt_zero_wn-dt_we_wn+dt_ns_wn-dt_non_wn)

            while offset_diff_we_ns < -0.5*t:
                offset_diff_we_ns += t
            while offset_diff_we_ns > 0.5*t:
                offset_diff_we_ns -= t
            #print("Delta O zy: {}".format(offset_diff_we_ns))

            offset_we_non = offset_we + \
                            0.5 * area_ratio_matrix[0][1][3]*t + \
                            dt_we_wn + \
                            getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns') / v - \
                            dt_non_wn - 0.5 * area_ratio_matrix[1][1][3]*t
            offset_we_non = offset_we_non % t

            ideal_dis_we = getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns')
            ideal_dis_ns = getIdealDis((1, 0), (1, 1), phase_ns, phase_non, t, 'we')
            real_dis_we = area_dis_matrix[1][1][1]
            real_dis_ns = area_dis_matrix[1][1][0]
            DT_we = getSRI(ideal_dis_we, real_dis_we, t, False) / v
            DT_ns = getSRI(ideal_dis_ns, real_dis_ns, t, False) / v
            #print("DT_we: {}, DT_ns: {}".format(DT_we, DT_ns))
            k_we = offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_we
            k_ns = -1 * offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_ns

            if DT_we or DT_ns:
                offset_diff_we_non = -1*k_we/(k_we+k_ns) * offset_diff_we_ns
            else:
                offset_diff_we_non = -0.5 * offset_diff_we_ns

            offset_diff_ns_non = offset_diff_we_ns + offset_diff_we_non
            offset_non = offset_we_non - offset_diff_we_non

            Dlambda_ns = (DT_ns + offset_diff_ns_non) / t
            Dlambda_we = (DT_we + offset_diff_we_non) / t
            bias_split = 2*(np.abs(Dlambda_ns) + np.abs(Dlambda_we))
            we_idx = 'WE' + str(phase_we)
            ns_idx = 'NS' + str(phase_ns)
            bias_split  +=  relative_df.loc[relative_df['cycle'] == t].iloc[0][we_idx] + \
                                    relative_df.loc[relative_df['cycle'] == t].iloc[0][ns_idx]
            if min_bias_split > bias_split:
                min_bias_split = bias_split
                optimal_seq = [phase_we, phase_ns, phase_non]

            print(bias_split)

print([t, min_bias_split, optimal_seq])


2.010965091657952
1.7659775641025641
7.141028904165219
0.3183239276118597
0.17597756410256404
0.0852414698358418
0.24084010111993331
0.30100689122772983
0.705800892209916
1.447451923076923
3.147180346598736
5.877031017934708
0.4520192307692306
0.5120192307692308
0.08266025641025643
0.03266025641025634
3.611944432480546
3.0568725122651057
0.9218910256410258
2.945800690335316
1.6226602564102566
2.214726134122289
0.45189102564102573
0.958992741104443
1.2550160256410257
1.0965589417539645
0.006874999999999812
0.05687500000000012
35.32558866055321
0.9034775641025642
0.7950160256410258
0.7350160256410255
1.4957852564102565
0.9304428513835622
0.32501602564102583
0.4434775641025642
[78.0, 0.006874999999999812, [4, 1, 2]]


全部周期

In [16]:
optimal_table = []
optimal_cols = ['cycle', 'bias_split', 'phase']

for idx, data in qualified_df.iterrows():
    t = data['cycle']
    seq_we = data['WE']
    seq_ns = data['NS']
    table_row = [t]

    dt_zero_ns = getDeltaT((0, 0), base_phase, t, 'ns')
    dt_zero_we = getDeltaT((0, 0), base_phase, t, 'we')
    dt_zero_wn = getDeltaT((0, 0), base_phase, t, 'wn')

    min_bias_split = float('inf')
    optimal_seq = []
    for phase_we in seq_we:
        dt_we_ns = getDeltaT((0, 1), phase_we, t, 'ns')
        dt_we_we = getDeltaT((0, 1), phase_we, t, 'we')
        dt_we_wn = getDeltaT((0, 1), phase_we, t, 'wn')

        offset_we = 0.5*area_ratio_matrix[0][0][3]*t + \
                    getIdealDis((0, 0), (0, 1), base_phase, phase_we, t, 'we')/v - \
                    0.5*area_ratio_matrix[0][1][3]*t
        offset_we = offset_we % t

        for phase_ns in seq_ns:
            dt_ns_ns = getDeltaT((1, 0), phase_ns, t, 'ns')
            dt_ns_we = getDeltaT((1, 0), phase_ns, t, 'we')
            dt_ns_wn = getDeltaT((1, 0), phase_ns, t, 'wn')

            for phase_non in range(6):
                dt_non_ns = getDeltaT((1, 1), phase_non, t, 'ns')
                dt_non_we = getDeltaT((1, 1), phase_non, t, 'we')
                dt_non_wn = getDeltaT((1, 1), phase_non, t, 'wn')

                offset_diff_we_ns = 0.5*(dt_zero_ns-dt_ns_ns+dt_ns_we-dt_non_we) - \
                                    0.5*(dt_zero_we-dt_we_we+dt_we_ns-dt_non_ns) + \
                                        (dt_zero_wn-dt_we_wn+dt_ns_wn-dt_non_wn)
                while offset_diff_we_ns < -0.5*t:
                    offset_diff_we_ns += t
                while offset_diff_we_ns > 0.5*t:
                    offset_diff_we_ns -= t

                offset_we_non = offset_we + \
                                0.5 * area_ratio_matrix[0][1][3]*t + \
                                dt_we_wn + \
                                getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns') / v - \
                                dt_non_wn - 0.5 * area_ratio_matrix[1][1][3]*t
                offset_we_non = offset_we_non % t

                ideal_dis_we = getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns')
                ideal_dis_ns = getIdealDis((1, 0), (1, 1), phase_ns, phase_non, t, 'we')
                real_dis_we = area_dis_matrix[1][1][1]
                real_dis_ns = area_dis_matrix[1][1][0]
                DT_we = getSRI(ideal_dis_we, real_dis_we, t, False) / v
                DT_ns = getSRI(ideal_dis_ns, real_dis_ns, t, False) / v
                #print("DT_we: {}, DT_ns: {}".format(DT_we, DT_ns))
                k_we = offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_we
                k_ns = -1 * offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_ns

                if DT_we or DT_ns:
                    offset_diff_we_non = -1*k_we/(k_we+k_ns) * offset_diff_we_ns
                else:
                    offset_diff_we_non = -0.5 * offset_diff_we_ns

                offset_diff_ns_non = offset_diff_we_ns + offset_diff_we_non
                offset_non = offset_we_non - offset_diff_we_non

                Dlambda_ns = (DT_ns + offset_diff_ns_non) / t
                Dlambda_we = (DT_we + offset_diff_we_non) / t
                bias_split = 2*(np.abs(Dlambda_ns) + np.abs(Dlambda_we))
                if min_bias_split > bias_split:
                    min_bias_split = bias_split
                    optimal_seq = [phase_we, phase_ns, phase_non]

    table_row.append(min_bias_split)
    table_row.append(optimal_seq)
    optimal_table.append(table_row)

optimal_df = pd.DataFrame(optimal_table, columns=optimal_cols)
optimal_df

optimal_data = optimal_df.loc[optimal_df['bias_split'] == optimal_df['bias_split'].min()]
optimal_data

,cycle,bias_split,phase
0,78.0,0.025545,"[4, 1, 2]"


绝对基准交叉口相位固定时的封装

In [17]:
def getOptimalData(base_phase):
    relative_table = []

    # clac relative benchmark
    ## clac bias split
    cols = ['cycle']
    cols += ['WE' + str(i) for i in range(6)]
    cols += ['NS' + str(i) for i in range(6)]
    cols.append('interval')

    for t in range(c_min, c_max+step, step):
        table_row = [t]
        interval = getInterval(t)

        for i in range(6):
            we_idea_dis = getIdealDis((0, 0), (0, 1), base_phase, i, t, 'we')
            we_dis = area_dis_matrix[0][1][0]
            we_bias = we_dis % interval
            we_diff = we_bias - we_idea_dis
            #print("idea: {}, real: {}, diff: {}".format(we_idea_dis, we_bias, we_diff))
            while np.abs(we_diff - interval) < np.abs(we_diff):
                we_diff -= interval
                #print(we_diff)
            while we_diff < 0 and np.abs(we_diff + interval) < np.abs(we_diff):
                we_diff += interval
                #print(we_diff)
            we_split = 0.5 * we_diff / interval

            table_row.append(we_split)

        for i in range(6):
            ns_idea_dis = getIdealDis((0, 0), (1, 0), base_phase, i, t, 'ns')
            ns_dis = area_dis_matrix[1][0][1]
            ns_bias = ns_dis % interval
            ns_diff = ns_bias - ns_idea_dis
            while np.abs(ns_diff - interval) < np.abs(ns_diff):
                ns_diff -= interval
            while ns_diff < 0 and np.abs(ns_diff + interval) < np.abs(ns_diff):
                ns_diff += interval
            ns_split = 0.5 * ns_diff / interval

            table_row.append(ns_split)

        table_row.append(interval)
        relative_table.append(table_row)

    relative_df = pd.DataFrame(relative_table, columns=cols)
    if base_phase == 0:
        relative_df.to_excel('relative.xlsx')

    ## filter

    qualified_table = []

    qualified_cols = ['cycle', 'WE', 'NS']

    for idx, data in relative_df.iterrows():
        table_row = [data['cycle']]
        phase_list = []
        for i in range(6):
            series_idx = 'WE' + str(i)
            if np.abs(data[series_idx]) <= relaxation:
                phase_list.append(i)
        if not phase_list:
            continue
        table_row.append(phase_list)

        phase_list = []
        for i in range(6):
            series_idx = 'NS' + str(i)
            if np.abs(data[series_idx]) <= relaxation:
                phase_list.append(i)
        if not phase_list:
            continue
        table_row.append(phase_list)

        qualified_table.append(table_row)

    qualified_df = pd.DataFrame(qualified_table, columns=qualified_cols)
    if base_phase == 0:
        qualified_df.to_excel('qualified.xlsx')

    # non-benchmark

    optimal_table = []
    optimal_cols = ['cycle', 'bias_split', 'phase', 'offset']

    for idx, data in qualified_df.iterrows():
        t = data['cycle']
        seq_we = data['WE']
        seq_ns = data['NS']
        table_row = [t]

        dt_zero_ns = getDeltaT((0, 0), base_phase, t, 'ns')
        dt_zero_we = getDeltaT((0, 0), base_phase, t, 'we')
        dt_zero_wn = getDeltaT((0, 0), base_phase, t, 'wn')

        min_bias_split = float('inf')
        optimal_seq = []
        optimal_offset = [0]
        for phase_we in seq_we:
            dt_we_ns = getDeltaT((0, 1), phase_we, t, 'ns')
            dt_we_we = getDeltaT((0, 1), phase_we, t, 'we')
            dt_we_wn = getDeltaT((0, 1), phase_we, t, 'wn')

            offset_we = 0.5*area_ratio_matrix[0][0][3]*t + \
                        getIdealDis((0, 0), (0, 1), base_phase, phase_we, t, 'we')/v - \
                        0.5*area_ratio_matrix[0][1][3]*t
            offset_we = offset_we % t

            for phase_ns in seq_ns:
                dt_ns_ns = getDeltaT((1, 0), phase_ns, t, 'ns')
                dt_ns_we = getDeltaT((1, 0), phase_ns, t, 'we')
                dt_ns_wn = getDeltaT((1, 0), phase_ns, t, 'wn')

                offset_ns = 0.5*area_ratio_matrix[0][0][3]*t + \
                            dt_zero_wn + \
                            getIdealDis((0, 0), (1, 0), base_phase, phase_ns, t, 'ns')/v - \
                            dt_ns_wn - \
                            0.5*area_ratio_matrix[1][0][3]

                for phase_non in range(6):
                    dt_non_ns = getDeltaT((1, 1), phase_non, t, 'ns')
                    dt_non_we = getDeltaT((1, 1), phase_non, t, 'we')
                    dt_non_wn = getDeltaT((1, 1), phase_non, t, 'wn')

                    offset_diff_we_ns = 0.5*(dt_zero_ns-dt_ns_ns+dt_ns_we-dt_non_we) - \
                                        0.5*(dt_zero_we-dt_we_we+dt_we_ns-dt_non_ns) + \
                                            (dt_zero_wn-dt_we_wn+dt_ns_wn-dt_non_wn)
                    while offset_diff_we_ns < -0.5*t:
                        offset_diff_we_ns += t
                    while offset_diff_we_ns > 0.5*t:
                        offset_diff_we_ns -= t

                    offset_we_non = offset_we + \
                                    0.5 * area_ratio_matrix[0][1][3]*t + \
                                    dt_we_wn + \
                                    getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns') / v - \
                                    dt_non_wn - 0.5 * area_ratio_matrix[1][1][3]*t
                    offset_we_non = offset_we_non % t

                    ideal_dis_we = getIdealDis((0, 1), (1, 1), phase_we, phase_non, t, 'ns')
                    ideal_dis_ns = getIdealDis((1, 0), (1, 1), phase_ns, phase_non, t, 'we')
                    real_dis_we = area_dis_matrix[1][1][1]
                    real_dis_ns = area_dis_matrix[1][1][0]
                    DT_we = getSRI(ideal_dis_we, real_dis_we, t, False) / v
                    DT_ns = getSRI(ideal_dis_ns, real_dis_ns, t, False) / v
                    #print("DT_we: {}, DT_ns: {}".format(DT_we, DT_ns))
                    k_we = offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_we
                    k_ns = -1 * offset_diff_we_ns / np.abs(offset_diff_we_ns) * DT_ns

                    if DT_we or DT_ns:
                        offset_diff_we_non = -1*k_we/(k_we+k_ns) * offset_diff_we_ns
                    else:
                        offset_diff_we_non = -0.5 * offset_diff_we_ns

                    offset_diff_ns_non = offset_diff_we_ns + offset_diff_we_non
                    offset_non = offset_we_non - offset_diff_we_non

                    Dlambda_ns = (DT_ns + offset_diff_ns_non) / t
                    Dlambda_we = (DT_we + offset_diff_we_non) / t

                    bias_split = 2*(np.abs(Dlambda_ns) + np.abs(Dlambda_we))

                    we_idx = 'WE' + str(phase_we)
                    ns_idx = 'NS' + str(phase_ns)
                    #print(relative_df)
                    bias_split  +=  relative_df.loc[relative_df['cycle'] == t].iloc[0][we_idx] + \
                                    relative_df.loc[relative_df['cycle'] == t].iloc[0][ns_idx]
                    #print(bias_split)

                    if min_bias_split > np.abs(bias_split):
                        min_bias_split = np.abs(bias_split)
                        optimal_seq = [phase_we, phase_ns, phase_non]
                        optimal_offset = [0, round(offset_we), round(offset_ns), round(offset_non)]

        table_row.append(min_bias_split)
        table_row.append(optimal_seq)
        table_row.append(optimal_offset)
        optimal_table.append(table_row)

    optimal_df = pd.DataFrame(optimal_table, columns=optimal_cols)
    optimal_data = optimal_df.loc[optimal_df['bias_split'] == optimal_df['bias_split'].min()]
    return optimal_data.iloc[0].to_numpy()
    #return optimal_df

getOptimalData(0)

array([90.0, 0.002624999999999822, list([4, 1, 3]), list([0, 21, 80, 10])],
      dtype=object)

In [18]:
optimal_scheme_cols = ['base phase', 'optimal cycle', 'bias_split', 'optimal phase combination', 'offset']
optimal_scheme_table = []

for i in range(6):
    row = list(getOptimalData(i))
    row.insert(0, i)
    optimal_scheme_table.append(row)

optimal_scheme_df = pd.DataFrame(optimal_scheme_table, columns=optimal_scheme_cols)
optimal_scheme_data = optimal_scheme_df.loc[optimal_scheme_df['bias_split'] == optimal_scheme_df['bias_split'].min()]
optimal_scheme_data.to_excel('optimal.xlsx')
optimal_scheme_data

,base phase,optimal cycle,bias_split,optimal phase combination,offset
1,1,88.0,0.000767,"[2, 0, 0]","[0, 27, -14, 39]"


# 输出

In [19]:
def getGreenTime(pos, start_time, phase, c, offset):
    """ Calculate the start time and end time of each direction
        green time will start with W direction

    Args:
        pos (tuple): (x, y) of intersection
        start_time (int): start time
        phase (int): phase number
        c (int): cycle
        offset (_int): offset

    Returns:
        list: list including 4 [start, end] lists
                sequence: SNEW
    """
    greentime_w = round(area_ratio_matrix[pos[0]][pos[1]][3]*c)
    greentime_e = round(area_ratio_matrix[pos[0]][pos[1]][2]*c)
    greentime_n = round(area_ratio_matrix[pos[0]][pos[1]][1]*c)
    greentime_s = round(area_ratio_matrix[pos[0]][pos[1]][0]*c)
    greentime_i = round((1 - sum(area_ratio_matrix[pos[0]][pos[1]]))*c)
    time_s = []
    time_n = []
    time_e = []
    time_w = []

    end = start_time + offset
    time_w.append(end)
    end += greentime_w + greentime_i
    time_w.append(end)
    if phase == 0:
        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)

        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)

        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)
    elif phase == 1:
        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)

        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)

        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)
    elif phase == 2:
        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)

        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)

        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)
    elif phase == 3:
        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)

        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)

        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)
    elif phase == 4:
        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)

        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)

        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)
    elif phase == 5:
        time_e.append(end)
        end += greentime_e
        end += greentime_i
        time_e.append(end)

        time_s.append(end)
        end += greentime_s
        end += greentime_i
        time_s.append(end)

        time_n.append(end)
        end += greentime_n
        end += greentime_i
        time_n.append(end)

    return [time_s, time_n, time_e, time_w]

In [20]:
phase_list_all = [optimal_scheme_data.iloc[0]['base phase']] + optimal_scheme_data.iloc[0]['optimal phase combination']
offset_final = optimal_scheme_data.iloc[0]['offset']
cycle = optimal_scheme_data.iloc[0]['optimal cycle']
offset_final

time_base = getGreenTime((0, 0), 0, phase_list_all[0], cycle, offset_final[0])
time_base

time_table = []
time_table.append(getGreenTime((0, 0), 0, phase_list_all[0], cycle, offset_final[0]))
time_table.append(getGreenTime((0, 1), 0, phase_list_all[1], cycle, offset_final[1]))
time_table.append(getGreenTime((1, 0), 0, phase_list_all[2], cycle, offset_final[2]))
time_table.append(getGreenTime((1, 1), 0, phase_list_all[3], cycle, offset_final[3]))

output_direction = ['E', 'S', 'W', 'N']
output_cols = ['intersection id']
for i in output_direction:
    output_cols.append('start - ' + i)
for i in output_direction:
    output_cols.append('end - ' + i)

output = []
for i in range(4):
    row = []
    '''
    if i == 0:
        row.append('(0, 0)')
    elif i == 1:
        row.append('(0, 1)')
    elif i == 2:
        row.append('(1, 0)')
    elif i == 3:
        row.append('(1, 1)')
    '''
    row.append(i)
    
    row.append(time_table[i][2][0])
    row.append(time_table[i][0][0])
    row.append(time_table[i][3][0])
    row.append(time_table[i][1][0])

    row.append(time_table[i][2][1])
    row.append(time_table[i][0][1])
    row.append(time_table[i][3][1])
    row.append(time_table[i][1][1])

    output.append(row)

output_df = pd.DataFrame(output, columns=output_cols)
output_df.to_excel('output.xlsx')
output_df

,intersection id,start - E,start - S,start - W,start - N,end - E,end - S,end - W,end - N
0,0,68,20,0,44,88,44,20,68
1,1,66,94,27,55,94,148,55,66
2,2,4,53,-14,27,27,74,4,53
3,3,97,119,39,108,108,161,97,119
